In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'nak_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2013.xlsx', '도개_2014.xlsx', '도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2013.xlsx', '신암_2014.xlsx', '신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx'], ['풍양_2013.xlsx', '풍양_2014.xlsx', '풍양_2015.xlsx', '풍양_2016.xlsx', '풍양_2017.xlsx', '풍양_2018.xlsx', '풍양_2019.xlsx', '풍양_2020.xlsx']]
data/nak/자동/도개_2013.xlsx
data/nak/자동/도개_2014.xlsx
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2013-01-01 00:00  ~  2020-12-31 23:00
data/nak/자동/신암_2013.xlsx
data/nak/자동/신암_2014.xlsx
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.9221 - disc_loss: 0.2960 - rmse: 0.3915 - val_loss: 0.3844
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3207 - disc_loss: 0.3025 - rmse: 0.4012 - val_loss: 0.4469
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7038 - disc_loss: 0.3180 - rmse: 0.4126 - val_loss: 0.4036
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4354 - disc_loss: 0.2895 - rmse: 0.3788 - val_loss: 0.3739
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7611 - disc_loss: 0.3162 - rmse: 0.3799 - val_loss: 0.3464
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.2987 - disc_loss: 0.3180 - rmse: 0.4278 - val_loss: 0.3946
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7709 - disc_loss: 0.2930 - rmse: 0.3838 - val_loss: 0.3864
Epoch 55/2000
1/1 [====================

Epoch 107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4792 - disc_loss: 0.2455 - rmse: 0.3345 - val_loss: 0.3353
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5290 - disc_loss: 0.2661 - rmse: 0.3090 - val_loss: 0.4324
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2206 - disc_loss: 0.2543 - rmse: 0.3290 - val_loss: 0.2981
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8151 - disc_loss: 0.2446 - rmse: 0.3668 - val_loss: 0.3447
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5355 - disc_loss: 0.2305 - rmse: 0.3247 - val_loss: 0.3082
Epoch 112/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.0027 - disc_loss: 0.2388 - rmse: 0.3300 - val_loss: 0.3511
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1746 - disc_loss: 0.2230 - rmse: 0.3441 - val_loss: 0.3520
Epoch 114/2000
1/1 

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1140 - disc_loss: 0.2234 - rmse: 0.2836 - val_loss: 0.3662
Epoch 166/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.7197 - disc_loss: 0.2297 - rmse: 0.2747 - val_loss: 0.2934
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4643 - disc_loss: 0.2144 - rmse: 0.2706 - val_loss: 0.3103
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6167 - disc_loss: 0.2297 - rmse: 0.3427 - val_loss: 0.3262
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8331 - disc_loss: 0.2330 - rmse: 0.3221 - val_loss: 0.3038
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5721 - disc_loss: 0.2354 - rmse: 0.3129 - val_loss: 0.3008
Epoch 171/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 4.8443 - disc_loss: 0.2242 - rmse: 0.3004 - val_loss: 0.2551
Epoch 172/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0759 - disc_loss: 0.2133 - rmse: 0.3936 - val_loss: 0.2882
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0052 - disc_loss: 0.2403 - rmse: 0.3607 - val_loss: 0.3148
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3872 - disc_loss: 0.2400 - rmse: 0.2544 - val_loss: 0.2551
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1125 - disc_loss: 0.2044 - rmse: 0.2609 - val_loss: 0.2921
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1388 - disc_loss: 0.2381 - rmse: 0.3760 - val_loss: 0.2930
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7320 - disc_loss: 0.2280 - rmse: 0.3239 - val_loss: 0.2848
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1118 - disc_loss: 0.2313 - rmse: 0.2718 - val_loss: 0.2716
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3891 - disc_loss: 0.1657 - rmse: 0.2485 - val_loss: 0.2981
Epoch 920/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9581 - disc_loss: 0.1709 - rmse: 0.2975 - val_loss: 0.2856
Epoch 921/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4475 - disc_loss: 0.1755 - rmse: 0.2386 - val_loss: 0.2716
Epoch 922/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7288 - disc_loss: 0.1672 - rmse: 0.2741 - val_loss: 0.2487
Epoch 923/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3890 - disc_loss: 0.1695 - rmse: 0.2176 - val_loss: 0.2435
Epoch 924/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1903 - disc_loss: 0.1660 - rmse: 0.2219 - val_loss: 0.2351
Epoch 925/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8585 - disc_loss: 0.1797 - rmse: 0.2572 - val_loss: 0.2521
Epoch 926/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6928 - disc_loss: 0.1661 - rmse: 0.2945 - val_loss: 0.2249
Epoch 978/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4347 - disc_loss: 0.1637 - rmse: 0.3086 - val_loss: 0.3055
Epoch 979/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4003 - disc_loss: 0.1674 - rmse: 0.2453 - val_loss: 0.2630
Epoch 980/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4774 - disc_loss: 0.1750 - rmse: 0.2387 - val_loss: 0.2700
Epoch 981/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1380 - disc_loss: 0.1688 - rmse: 0.2396 - val_loss: 0.2741
Epoch 982/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4146 - disc_loss: 0.1694 - rmse: 0.2482 - val_loss: 0.2578
Epoch 983/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0513 - disc_loss: 0.1827 - rmse: 0.3548 - val_loss: 0.2115
Epoch 984/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7968 - disc_loss: 0.1642 - rmse: 0.2567 - val_loss: 0.2804
Epoch 1036/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7798 - disc_loss: 0.1714 - rmse: 0.2413 - val_loss: 0.2709
Epoch 1037/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9225 - disc_loss: 0.1767 - rmse: 0.2858 - val_loss: 0.3237
Epoch 1038/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6995 - disc_loss: 0.1734 - rmse: 0.2934 - val_loss: 0.2312
Epoch 1039/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9069 - disc_loss: 0.1608 - rmse: 0.2622 - val_loss: 0.2344
Epoch 1040/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2098 - disc_loss: 0.1614 - rmse: 0.3004 - val_loss: 0.2529
Epoch 1041/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0026 - disc_loss: 0.1604 - rmse: 0.2319 - val_loss: 0.2520
Epoch 1042/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8759 - disc_loss: 0.1690 - rmse: 0.2234 - val_loss: 0.3178
Epoch 1094/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2448 - disc_loss: 0.1667 - rmse: 0.2654 - val_loss: 0.2672
Epoch 1095/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5903 - disc_loss: 0.1484 - rmse: 0.2533 - val_loss: 0.2202
Epoch 1096/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9360 - disc_loss: 0.1645 - rmse: 0.2162 - val_loss: 0.3019
Epoch 1097/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7745 - disc_loss: 0.1631 - rmse: 0.2547 - val_loss: 0.2951
Epoch 1098/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1564 - disc_loss: 0.1704 - rmse: 0.2448 - val_loss: 0.2403
Epoch 1099/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2830 - disc_loss: 0.1688 - rmse: 0.2697 - val_loss: 0.2619
Epoch 1100/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4659 - disc_loss: 0.1631 - rmse: 0.2628 - val_loss: 0.2939
Epoch 1152/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7947 - disc_loss: 0.1682 - rmse: 0.2361 - val_loss: 0.2318
Epoch 1153/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8875 - disc_loss: 0.1646 - rmse: 0.2337 - val_loss: 0.3186
Epoch 1154/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1386 - disc_loss: 0.1714 - rmse: 0.2762 - val_loss: 0.2574
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2428 - disc_loss: 0.1685 - rmse: 0.2442 - val_loss: 0.2523
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9100 - disc_loss: 0.1592 - rmse: 0.2165 - val_loss: 0.2490
Epoch 1157/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7835 - disc_loss: 0.1641 - rmse: 0.2680 - val_loss: 0.2601
Epoch 1158/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [32]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(70128, 36)
toc


In [33]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (49089, 36) val_df.shape :  (14026, 36) test_df.shape: (7013, 36)


In [34]:
#fake_df.shape[0]/8760

In [35]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [38]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [39]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [40]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/nak_auto_13/toc/


In [41]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [42]:
rnn_epochs

20

## 모델 학습

In [43]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.6533 - mean_absolute_error: 0.5571 - nse: 0.1229
Epoch 00001: val_nse improved from -inf to 0.28594, saving model to save/best_model.h5
10/10 [==============================] - 10s 1s/step - loss: 0.6533 - mean_absolute_error: 0.5571 - nse: 0.1229 - val_loss: 0.4640 - val_mean_absolute_error: 0.5220 - val_nse: 0.2859
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.3820 - mean_absolute_error: 0.4235 - nse: 0.5552
Epoch 00002: val_nse improved from 0.28594 to 0.62458, saving model to save/best_model.h5
10/10 [==============================] - 10s 977ms/step - loss: 0.3820 - mean_absolute_error: 0.4235 - nse: 0.5552 - val_loss: 0.2449 - val_mean_absolute_error: 0.3406 - val_nse: 0.6246
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.2762 - mean_absolute_error: 0.3625 - nse: 0.6599
Epoch 00003: val_nse improved from 0.62458 to 0.64720, saving model to save/best_model.h5
10/10 [

Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.1942 - mean_absolute_error: 0.3013 - nse: 0.7428
Epoch 00023: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 982ms/step - loss: 0.1942 - mean_absolute_error: 0.3013 - nse: 0.7428 - val_loss: 0.3628 - val_mean_absolute_error: 0.4294 - val_nse: 0.4382
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.1798 - mean_absolute_error: 0.2926 - nse: 0.7888
Epoch 00024: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 979ms/step - loss: 0.1798 - mean_absolute_error: 0.2926 - nse: 0.7888 - val_loss: 0.4054 - val_mean_absolute_error: 0.4520 - val_nse: 0.3709
Epoch 25/50
10/10 [==============================] - ETA: 0s - loss: 0.2010 - mean_absolute_error: 0.2994 - nse: 0.7728
Epoch 00025: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 983ms/step - loss: 0.2010 - mean_absolute_error: 0.2994 - nse: 0.7728 - v

Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.1354 - mean_absolute_error: 0.2560 - nse: 0.8196
Epoch 00046: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 980ms/step - loss: 0.1354 - mean_absolute_error: 0.2560 - nse: 0.8196 - val_loss: 0.5034 - val_mean_absolute_error: 0.4901 - val_nse: 0.2219
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.1401 - mean_absolute_error: 0.2594 - nse: 0.8341
Epoch 00047: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 971ms/step - loss: 0.1401 - mean_absolute_error: 0.2594 - nse: 0.8341 - val_loss: 0.6361 - val_mean_absolute_error: 0.5404 - val_nse: 0.0153
Epoch 48/50
10/10 [==============================] - ETA: 0s - loss: 0.1436 - mean_absolute_error: 0.2657 - nse: 0.8253
Epoch 00048: val_nse did not improve from 0.64720
10/10 [==============================] - 10s 977ms/step - loss: 0.1436 - mean_absolute_error: 0.2657 - nse: 0.8253 - v

## core / window.py / 

In [44]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  3.0531461
predict_day :  5
------------------------
0.27329153 0.16610453 0.40755922
nse3 :  [0.5235018]
pbias3 :  [1.1429733]
save/nak/models/nak_auto_13/toc/
year : 2013 ~ 2020
run :  range(0, 1)
target :  toc
length :  70128
train=7, test=1, val=2
---------------
[0.5235018]
[1.1429733]


In [45]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
